In [1]:
from source.library.scraping import scrape_all_urls#, scrape_urls

# This is needed because openai.text_completion calls asynchronous functions but
# Jupyter is already running its own event loop.
import nest_asyncio
nest_asyncio.apply()

In [2]:
target_url = 'https://python.langchain.com'
results = scrape_all_urls(target_url)
print(len(results))

/code/source/library/scraping.py:86: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


1238


In [3]:
results = list(results)
results[0]

('https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/twitter.html',
 '\n\n<!DOCTYPE html>\n\n\n<html lang="en" >\n\n  <head>\n    <meta charset="utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="generator" content="Docutils 0.17.1: http://docutils.sourceforge.net/" />\n\n    <title>Twitter &#8212; 🦜🔗 LangChain 0.0.155</title>\n  \n  \n  \n  <script data-cfasync="false">\n    document.documentElement.dataset.mode = localStorage.getItem("mode") || "";\n    document.documentElement.dataset.theme = localStorage.getItem("theme") || "light";\n  </script>\n  \n  <!-- Loaded before other Sphinx assets -->\n  <link href="../../../../_static/styles/theme.css?digest=12da95d707ffb74b382d" rel="stylesheet" />\n<link href="../../../../_static/styles/bootstrap.css?digest=12da95d707ffb74b382d" rel="stylesheet" />\n<link href="../../../../_static/styles/pydata-sphinx-theme.css?digest=12da95d707ffb74b382d" rel="stylesheet" />

In [4]:
urls_htmls = [x for x in results if x[0].endswith('/') or x[0].endswith('.html')]
print(len(urls_htmls))

# urls_to_scrape = [x for x in urls if x.endswith('/') or x.endswith('.html')]
# print(len(urls_to_scrape))
# htmls = scrape_urls(urls=urls_to_scrape)
# print(len(htmls))

754


In [5]:
# htmls[0]

In [6]:
from bs4 import BeautifulSoup
import numpy as np
import chromadb

client = chromadb.Client()

collection = client.create_collection('sample_collection')



documents = []
metadatas = []
ids = []

for url, html in urls_htmls:
    soup = BeautifulSoup(html, 'html.parser')
    for p in soup.find_all('p'):
        text = p.get_text().strip()
        if len(text.split(' ')) > 5:  # TODO: what should this be?
            documents.append(text.strip())
            metadatas.append({'url': url})

collection.add(
    documents=documents, # we embed for you, or bring your own
    metadatas=metadatas, # filter on arbitrary metadata!
    ids=[str(x) for x in range(len(documents))], # must be unique for each doc 
)

print(np.mean([len(x) for x in documents]))
print(np.max([len(x) for x in documents]))

Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


92.81811085089774
3285


In [7]:
print(documents[0:100])

['This loader fetches the text from the Tweets of a list of Twitter users, using the tweepy Python package.\nYou must initialize the loader with your Twitter API token, and you need to pass in the Twitter username you want to extract.', 'Last updated on May 02, 2023.', 'This notebook goes over adding memory to an Agent. Before going through this notebook, please walkthrough the following notebooks, as this will build on top of both of them:', 'Adding memory to an LLM Chain', 'In order to add a memory to an agent we are going to the the following steps:', 'We are going to create an LLMChain with memory.', 'We are going to use that LLMChain to create a custom Agent.', 'For the purposes of this exercise, we are going to create a simple custom Agent that has access to a search tool and utilizes the ConversationBufferMemory class.', 'Notice the usage of the chat_history variable in the PromptTemplate, which matches up with the dynamic key name in the ConversationBufferMemory.', 'We can now 

In [8]:
results = collection.query(
    query_texts="What is a document loader?",
    n_results=5,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)
print(len(results['documents'][0]))

5


In [9]:
results

{'ids': [['1901', '171', '2050', '1907', '2042']],
 'embeddings': None,
 'documents': [['All different types of document loaders.',
   'The following document loaders are provided:',
   'Loader that uses unstructured to load word documents.',
   'Each document represents one Document.\nThe loader converts the original PDF format into the text.',
   'Loader that uses unstructured to load files.']],
 'metadatas': [[{'url': 'https://python.langchain.com/en/latest/reference/modules/document_loaders.html'},
   {'url': 'https://python.langchain.com/en/latest/modules/indexes/document_loaders.html'},
   {'url': 'https://python.langchain.com/en/latest/reference/modules/document_loaders.html'},
   {'url': 'https://python.langchain.com/en/latest/reference/modules/document_loaders.html'},
   {'url': 'https://python.langchain.com/en/latest/reference/modules/document_loaders.html'}]],
 'distances': [[0.2744857966899872,
   0.33620381355285645,
   0.48186448216438293,
   0.5686023831367493,
   0.5878